# Train LSTM model

Refer to https://github.com/keras-team/keras/blob/master/examples/lstm_text_generation.py

In [3]:
from keras.models        import Sequential
from keras.layers        import Input, Dense, Dropout
from keras.layers        import Embedding
from keras.layers        import LSTM
from keras               import backend as K
from keras.preprocessing import sequence
import tensorflow as tf

import pickle
import numpy      as np

In [4]:
# Set to cpu mode
num_cores = 1
num_CPU = 1
num_GPU = 0
config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

In [6]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4533133700113221406
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 218890240
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8330744733997129862
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:03:00.0, compute capability: 3.5"
]


In [7]:
latent_dim = 1024

## Load data

In [5]:
X_train, y_train, index_to_word, word_to_index = pickle.load(open('trainingData.pkl', 'rb'))

In [6]:
print('X_train shape: {}, Y_train shape: {}'.format(X_train.shape, y_train.shape))

X_train shape: (48975,), Y_train shape: (48975,)


In [7]:
print(X_train[0])
print(y_train[0])
print(index_to_word[0:10])
print(list(word_to_index.keys())[0:10])

[4317, 1594, 6824, 1632, 1366, 7488, 511, 5967, 1153, 4001, 1456, 76, 4742, 2986, 642, 1426, 6078, 7488, 4299, 1391, 3997, 6828, 7056, 76, 6310, 5442, 6379, 2234, 3038, 7488, 7010, 5777, 6314, 492, 5486, 76, 152, 2434, 6803, 1454, 6746, 7488, 6998, 3516, 509, 4742, 3918]
[1594, 6824, 1632, 1366, 7488, 511, 5967, 1153, 4001, 1456, 76, 4742, 2986, 642, 1426, 6078, 7488, 4299, 1391, 3997, 6828, 7056, 76, 6310, 5442, 6379, 2234, 3038, 7488, 7010, 5777, 6314, 492, 5486, 76, 152, 2434, 6803, 1454, 6746, 7488, 6998, 3516, 509, 4742, 3918, 76]
[' ', '(', ')', ',', '-', '.', '0', '1', '2', '3']
['媮', '葌', '辞', '兕', '篥', '丝', '省', '互', '褕', '土']


In [86]:
input_characters = sorted(list(word_to_index))
target_characters = sorted(list(word_to_index))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in X_train])
max_decoder_seq_length = max([len(txt) for txt in y_train])

print('Number of samples:', len(X_train))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 48975
Number of unique input tokens: 7492
Number of unique output tokens: 7492
Max sequence length for inputs: 2445
Max sequence length for outputs: 2445


## Define model

In [54]:
from keras.callbacks import LambdaCallback
from keras.models        import Sequential
from keras.layers        import Input, Dense, Dropout
from keras.layers        import Embedding
from keras.layers        import LSTM
from keras.optimizers import RMSprop
from keras               import backend as K
from keras.preprocessing import sequence

import random

In [49]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(input_characters))))
model.add(Dense(len(input_characters), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [107]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 128)               3901952   
_________________________________________________________________
dense_4 (Dense)              (None, 7492)              966468    
Total params: 4,868,420
Trainable params: 4,868,420
Non-trainable params: 0
_________________________________________________________________
None


# Start training

### Create data generator

In [45]:
import keras
from keras.callbacks import LambdaCallback
from keras.models        import Sequential
from keras.layers        import Input, Dense, Dropout
from keras.layers        import Embedding
from keras.layers        import LSTM
from keras.optimizers import RMSprop
from keras               import backend as K
from keras.preprocessing import sequence
import tensorflow as tf

import random
import pickle
import numpy      as np

In [50]:
maxlen = 40
batch_size = 128

In [140]:
# After load data
class DataGenerator(keras.utils.Sequence):
    """Generates data for Keras."""
    def __init__(self, X_train=X_train, y_train=y_train, maxlen=maxlen, n_classes=len(input_characters), shuffle=True, 
                batch_size=batch_size):
        """Initialization.
        
        Args:
        """
        self.X_train = X_train
        self.y_train = y_train
        self.maxlen = maxlen
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.on_epoch_end()

    def __len__(self):
        """Denotes the number of batches per epoch."""
        return int(np.floor(len(self.X_train) / self.batch_size))

    def __getitem__(self, index):
        """Generate one batch of data."""
        # Randomly sample a sentence length
        sentence_length = np.random.randint(self.maxlen - 1) + 1  # Guarantee pick at least 1 word
        
        x = []
        y = []
        for n in range(self.batch_size):
            # Randomly sample a poem
            poem_id = np.random.randint(len(self.X_train))
            poem = self.X_train[poem_id]
            
            # Create sentence and next_char
            sentence = poem[0:sentence_length]
            
            # Vectorize
            x_temp = np.zeros((sentence_length, self.n_classes))
            y_temp = np.zeros(self.n_classes)
            for t, char in enumerate(sentence):
                x_temp[t, char] = 1
            if len(self.y_train[poem_id]) >= sentence_length:
                y_temp[self.y_train[poem_id][sentence_length - 1]] = 1
            else:
                y_temp[5] = 1  # Mark as "."
            x.append(x_temp)
            y.append(y_temp)
        
        X = np.array(x, dtype=np.bool)
        y = np.array(y, dtype=np.bool)
        return X, y

    def on_epoch_end(self):
        """Updates indexes after each epoch."""
        # self.indexes = np.arange(len(self.img_files))
        # if self.shuffle == True:
        #    np.random.shuffle(self.indexes)
        pass
    
train_datagen = DataGenerator()

### Build model

In [80]:
# build the model: a single LSTM
print('Build model...')
model2 = Sequential()
model2.add(LSTM(128, input_shape=(None, len(input_characters))))
model2.add(Dense(len(input_characters), activation='softmax'))

optimizer = RMSprop(lr=0.01)
model2.compile(loss='categorical_crossentropy', optimizer=optimizer)

print(model2.summary())

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               3901952   
_________________________________________________________________
dense_2 (Dense)              (None, 7492)              966468    
Total params: 4,868,420
Trainable params: 4,868,420
Non-trainable params: 0
_________________________________________________________________
None


In [420]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end2(epoch, _, input_sentence="风停", temperature=1.3, maxlen=40):
    # Function invoked at end of each epoch. Prints generated text.
    # Recurrent for maxlen
    cur_len = len(input_sentence)

    # Initiate
    input_x_index = [word_to_index[i] for i in input_sentence]
    input_x = np.zeros((1, maxlen, len(input_characters)))
    for i, index in enumerate(input_x_index):
        input_x[0, i, index] = 1

    while cur_len < maxlen:
        preds = model2.predict(input_x[:, 0:cur_len, :], verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = index_to_word[next_index]
        input_sentence += next_char
        input_x[0, cur_len, next_index] = 1
        cur_len += 1
        
    print("Epoch {}, output: {}".format(epoch, input_sentence))

print_callback = LambdaCallback(on_epoch_end=on_epoch_end2)

# model2.fit(x, y,
#           batch_size=128,
#           epochs=1,
#           callbacks=[print_callback])
model2.fit_generator(train_datagen, epochs=100, use_multiprocessing=True, verbose=1, 
                    callbacks=[print_callback])

# model2.save("40_words.h5")

Epoch 1/100
382/382 [==============================] - 1998s 5s/step - loss: 7.7780
Epoch 0, output: 风停居主槛日风儿.日日叶临曲载处女.臣得中席留玉舟云狂张门.昔言时雪书火逐沧日.
Epoch 2/100
382/382 [==============================] - 1923s 5s/step - loss: 7.7111
Epoch 1, output: 风停带苏雨初浦子.喜得日马日西暖得日能者千得.峰门日八花散。何日火田过幕以无木江
Epoch 3/100
382/382 [==============================] - 1773s 5s/step - loss: 7.9446
Epoch 2, output: 风停头遗放尽高依小客客..王多岁.珠明玉畔月成强回.隔绿朱三.可五笑多子雁中野来
Epoch 4/100
382/382 [==============================] - 1753s 5s/step - loss: 7.8403
Epoch 3, output: 风停千去苋，香栖犹雁书.灵浪复语晚绿书寒存，同，雪别双思幽横合浪，萧国忆识通终乱
Epoch 5/100
382/382 [==============================] - 1798s 5s/step - loss: 7.0512
Epoch 4, output: 风停苍儿阴车稍世送遗龙乐渡还云甘妆黛.三游..。水览终官今烻江。凤帷繁水烛旗江风
Epoch 6/100
382/382 [==============================] - 1790s 5s/step - loss: 6.3711
Epoch 5, output: 风停长二手千满何一乘边酒牛满台水前父静乱堪.老，终指恨情天.五目春.独立一小阑重
Epoch 7/100
382/382 [==============================] - 1804s 5s/step - loss: 6.2437
Epoch 6, output: 风停拥玉霞笼侵，不僧草安落江临诗正垂陵故群。应女暮时日梦深欣回.永.中杯魂...
Epoch 

Process ForkPoolWorker-59:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [452]:
model2.fit_generator(train_datagen, epochs=100, use_multiprocessing=True, verbose=1, 
                    callbacks=[print_callback])

Epoch 1/100
382/382 [==============================] - 897s 2s/step - loss: 6.2833
Epoch 0, output: 风停已相重，谢树爱上依。客路丹鹤齐，题玉便不中。圣歌啼断静，臣共奇昼许。斋若驱暮
Epoch 2/100
382/382 [==============================] - 914s 2s/step - loss: 6.0667
Epoch 1, output: 风停西风赏，香晚门复才。公女恩阁影，翠景惟常当。谢水天衔傍.桥，公道愁及草。晓妾
Epoch 3/100
382/382 [==============================] - 942s 2s/step - loss: 5.9587
Epoch 2, output: 风停住映得上月，多庐剑影著关在。还香大事在封楼，远室江边往我夫。帘行江边独台阙，
Epoch 4/100
382/382 [==============================] - 960s 3s/step - loss: 5.9300
Epoch 3, output: 风停红窗向陵得，飘衡游水多四啼。深入酒暖伴细我，休树独满行。桃源衔幽愁洞上，山曾
Epoch 5/100
382/382 [==============================] - 962s 3s/step - loss: 5.8818
Epoch 4, output: 风停池望离遥瞻，月沙礼残北来子。鸣下鸿步容迎龙非不吹远尘乌云薄金高酒，水面三军金
Epoch 6/100
382/382 [==============================] - 920s 2s/step - loss: 6.1311
Epoch 5, output: 风停不得车马欲贵地，古今臣今听圣花。城信茫云犹带烛，乍因客出东征。安边衔微侣此此
Epoch 7/100
382/382 [==============================] - 947s 2s/step - loss: 6.1048
Epoch 6, output: 风停露华岳臣尺，帐马南朝青衣。照雪幽井罢亲志，金端欲事春下存。院南广台狂筵孤，岳
Epoch 8/100
3

Process ForkPoolWorker-91:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [464]:
model2.fit_generator(train_datagen, epochs=100, use_multiprocessing=True, verbose=1, 
                    callbacks=[print_callback])

Epoch 1/100
382/382 [==============================] - 938s 2s/step - loss: 5.8316
Epoch 0, output: 风停在扉贵，好信高花枝。以间一五镜，为官何异休。野酒此里石，青山坳外怜。江泉厌独
Epoch 2/100
382/382 [==============================] - 937s 2s/step - loss: 5.6387
Epoch 1, output: 风停连江色，飞鞭两中上。客公亦已尘，今日沼城席。有觞湖十水，北忆弹鸣雨。烟苔还如
Epoch 3/100
382/382 [==============================] - 954s 2s/step - loss: 6.1304
Epoch 2, output: 风停芳树最秋雾，一半浦衣夫怀家。与人看处罢难莫残未尝树童子平禅筵夷.......
Epoch 4/100
382/382 [==============================] - 993s 3s/step - loss: 5.9577
Epoch 3, output: 风停之戈老华过，声叹更谁明月时。病时珠霞开时盖，兰室风数对鸡声。高兴徒眼黄化近，
Epoch 5/100
382/382 [==============================] - 983s 3s/step - loss: 5.9488
Epoch 4, output: 风停千里气，帝僧映山中。月顾高两凭，归林问自谁。露过将依至，园中须初木。古年安香
Epoch 6/100
382/382 [==============================] - 1095s 3s/step - loss: 6.1103
Epoch 5, output: 风停又来似长条，会功天来我好影雨年。寂寥度身无情远，横枝桂人久欲过。见年过事相送
Epoch 7/100
382/382 [==============================] - 962s 3s/step - loss: 6.0970
Epoch 6, output: 风停故国买，晚月更野公。长空落秋上，贫离雨倾苔。步月知其征生翠，须在从三求。江洒
Epoch 8/100


Process ForkPoolWorker-100:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [489]:
model2.fit_generator(train_datagen, epochs=1000, use_multiprocessing=True, verbose=1, 
                    callbacks=[print_callback])

Epoch 1/1000
382/382 [==============================] - 940s 2s/step - loss: 6.1563
Epoch 0, output: 风停绿九知，人物相何明。有何无处时，作梁中俱岛。阴竹其事雪，王城潭下树。谁道又为
Epoch 2/1000
382/382 [==============================] - 938s 2s/step - loss: 6.0238


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 1, output: 风停生连缘落解，后户凌露洞庭。迥不无处怀知树水尚却。云发闲急手怀语，不望孤房却向
Epoch 3/1000
382/382 [==============================] - 946s 2s/step - loss: 6.3711
Epoch 2, output: 风停遗峰去前人，三湘离说莫朝秋。谁将限秋伤作出，莫时岁月与青山。向雨身是三苦浦，
Epoch 4/1000
382/382 [==============================] - 944s 2s/step - loss: 6.1148
Epoch 3, output: 风停闲食远几香，侵朝游子寒浪人。楚阳旧平酒，长沙坐别角。终是月开至，事郊和成鹤。
Epoch 5/1000
382/382 [==============================] - 960s 3s/step - loss: 6.2537
Epoch 4, output: 风停说中堂路新，公地沾拂愿明来。西帆暮花是松入，和酒鹿多是新染。将生陪飘他里度，
Epoch 6/1000
382/382 [==============================] - 975s 3s/step - loss: 6.1900
Epoch 5, output: 风停双群寄时吟，金楼上地我能前。堂轮不泉所交水，玉来得长绿水阴。鸡峰江树在庙斜，
Epoch 7/1000
382/382 [==============================] - 963s 3s/step - loss: 6.2288
Epoch 6, output: 风停初劳事有儿，故乡向已意六夜。起多有鸟杯伤粉，心情烟色玉山期。名期放书隔月走，
Epoch 8/1000
382/382 [==============================] - 977s 3s/step - loss: 6.1676
Epoch 7, output: 风停如香高头冥，白翠风结醉世开。门中何得，西月怨家夫门。闻带还知》）斋还明。胡城
Epoch 9/1000
382/382 [==============================] - 1281s 3s/step - loss: 6.1909
Epoch

Process ForkPoolWorker-189:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


KeyboardInterrupt: 

In [491]:
# model2.save("40_words.h5")
# model2.save("40_words_epoch_59.h5")
# model2.save("40_words_epoch_124.h5")
# model2.save("40_words_epoch_145.h5")

## Play with the model

In [81]:
on_epoch_end2(1, _)

Epoch 1, output: 风停螭静偈漼迮鲰藖弱喎台慠麑勩芷涨儿蹢樛钉沾煞沱嫖笮谩潭拯硩诬殢螵懹碇揽壝袜候髟


In [83]:
on_epoch_end2(1, _)

Epoch 1, output: 风停瑱瘗濴昔烟央高章观入烟灰椤花陛昔殿禹劳俯庆山，忻淳绯原云崖眷惊方驻长河古云奕


In [95]:
on_epoch_end2(1, _)

Epoch 1, output: 风停高原仙禹非新访成树半穆霞戟劳星夷绮树辉新主志除总榆峰何全年基逐。劳燕驰霜日岩


In [148]:
on_epoch_end2(1, _)

Epoch 1, output: 风停陵川日。，丽去如。。帐。功。日。。。。。，。为旧。。庭。卷双。花雁。。外。。


In [150]:
on_epoch_end2(1, _)

Epoch 1, output: 风停。开观五世已。有，回四边花边吹五云随已罗可有。一鸣开竹深四全景殿庭马吹世年词


In [155]:
on_epoch_end2(1, _, temperature=1.2)

Epoch 1, output: 风停馆人。，月亦秀道肃移开翻，时流尽，为清，已月，，金秋。树光河月不半开有黄家在


In [157]:
on_epoch_end2(1, _)

Epoch 1, output: 风停来，一人今四静多一。，流思发门落圣发，已已处古首长乡下城轻垂。，往自里佩书天


In [167]:
on_epoch_end2(1, _, temperature=1.2)

Epoch 1, output: 风停雪步夜深，吹待非。。结独景下。。汉易，。在与车春风。落月待散，正流罢人玉处。


In [173]:
on_epoch_end2(1, _, temperature=1.3)

Epoch 1, output: 风停阳在下重。龙在有日。天林，已枝。阙隐迹茂。一弦此风今。气。花天来夜色。家映此


In [313]:
on_epoch_end2(2, _, temperature=1.6)

Epoch 2, output: 风停罕烨川动马，岂上五君似有空，。，林门城中喜书。乐阳凉霞。地，玉林月径车。章似


In [352]:
on_epoch_end2(3, _, temperature=1.6)

Epoch 3, output: 风停忍一风今君旧，者音处万一中水千鹤回一开一田落音尘。清光岂云翠水风，月白。白斜


In [389]:
on_epoch_end2(5, _, temperature=1.6)

Epoch 5, output: 风停啐幽口青山钟时波中下清然带月去，那又王经大春虚主叶童临在经郭无水独竹烟在栖同


In [403]:
on_epoch_end2(5, _, temperature=1.2)

Epoch 5, output: 风停悠传迷未知。.经闻云。心底扇，一风得曾好迷俗相.是忽.虚.当接霞穷怅见含得结


In [419]:
on_epoch_end2(5, _, temperature=1.5)

Epoch 5, output: 风停号姻莫僧迹伴忽篱南白销通.来文时瞻枝衣.金流在摇绿雨胡君虚.思长忽外尘孤贤见


In [425]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100)

Epoch 32, output: 风停去音道，应被与岁墙。事灭紫积华，滴自已舞香。不得香惆西，后折衔纵歌。绮成处此看，要下人未心。与由香惆目，折物一殊方。秋宿北天水，逐名客沙华。人间齐风井，人家


In [426]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100)

Epoch 32, output: 风停白露步后辞，当道多对何处枕。见荒对元升雨酒，应怜孙父水岭花。万日象笼多二龙，游教无白满须溪。开云向高仙夕性，下知列先暮鹤游。似堪巷与须飞梦，长长鹤欲阳长西。官上家入悠花乱，可怜蝉行花情忧。前将去馀


In [427]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100)

Epoch 32, output: 风停难疏代百战，心路鱼冠别君音。云母云火雕童酒，何须丝白一目门。九庭学先风烟事，黄东光方升埃去。岸上不成大江月，白月岩中上静明。闲又云石结春去，洲界人间多裳唯，头冠去尽学藏齐。轻玉不馀思能亦异音，山母


In [428]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100)

Epoch 32, output: 风停一飞笛，明月带风歌。萧向行弄晓，惊色须龙云。新尽休驾里，几月弄青田。正往楚翠无酒国，虫飞书潭冷山逐竹飞。东西为天衣地冷，玉此因深是荒郡。路自朝照华时客，若闻心下东万君。木金雪外歌多得，归云皆成桂袖


In [429]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100)

Epoch 32, output: 风停近钱公待传赤景，未由飘月明月儿。不知力且手臣息，归树秦山秋阳风。结来草殿苦别花，有舟罢池天下得，燕上与流犹待送。江上海台传枝马，在手若应树重非。西月云中下阙中。暗上风游力道水。清管人间为前居，积云


In [431]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100, input_sentence="黑云压城城欲摧，")

Epoch 32, output: 黑云压城城欲摧，几共几泪满师墙。被知尽有唯观意，愁时应去如今几.....................................................................


In [432]:
on_epoch_end2(32, _, temperature=1.2, maxlen=100, input_sentence="今夕是何夕，")

Epoch 32, output: 今夕是何夕，既去双大斜。远唱千万水，双暖慵吹汉。道弟上子洞，扬为自斜书。红然邻空暖，羽漏承不叹。飘龙忽已力，寒堂问池华。暮火谁忍雁，竹广是通霞。雨县平高一，里，犹共我。野云浮酒乱，惊谷落平枝。无限似先


In [433]:
on_epoch_end2(32, _, temperature=1.1, maxlen=100, input_sentence="今夕是何夕，")

Epoch 32, output: 今夕是何夕，众去过绣上。夜深松尘易，节应异落上。身宦无津华，程复越月开。不知千馀动，岂为东门德。落月上方水，凄映过风城。野须.正水，阳冷竟自下。欲知层断恨，新涧深夜吟园风。要交沧洲色，分用弄云门。光生


In [435]:
on_epoch_end2(32, _, temperature=1.5, maxlen=300, input_sentence="今夕是何夕，")

Epoch 32, output: 今夕是何夕，动口下天情。最闻千岩满月月，前羽红少摇东府。苍龙见丹宴春去，愁来应折多里闻。作歌开玉出气临。闲向江云连泥如。闲山河日静玉峰，哀沙日下禅津郎，此摇闲多天边急，虎风军身千忧依。府来入槛胡复地，边日分鸟柳鱼阁。二年还禅千里日，纱上孤文遍暮秦。朝日经人新看去，流影白发柳枝毛。说倚漫泪御信栽。吏，照携窗酒礼，边树.殿学端馀。尘馆必扬象难处，不鸟曲花别时气。披鸟井锦罗路宫，唯有故江卧溪洲。京想.涧堂一急，独须朱石感后山。落色士人牵万通，诗此啼下已间船。几上鸾空连芜动明暮枯雁还终手，访眼长莲落重容。西西衣裳久帝久移静眼悠管却千梅结懒照凄。虽思数壑宿不影，岁交路入马叶浅。辞山晚长才晓代燕重我苔白旧


In [436]:
on_epoch_end2(32, _, temperature=1.3, maxlen=300, input_sentence="今夕是何夕，")

Epoch 32, output: 今夕是何夕，锦食千重归。一径断太净，诗雨两地魂。翠云南台语，迥凄沙金落。啼语不知坐，台自异多听。秋朝绝袖马，何成掩玄深。似对共通解，过皇群非名。晨然此中吏，容马又庙池里眼，阴中城更岸。岳.悲先翠留乐，好见带月栽。广岩洞树尽，微涛何思色，上径汉月发。公飞万重雪，开山雨转长。时在华桂香，湖上拂平子。游过旧先未.仙，幽山天寒三。登春罗丹始，醉情自听馀。吟在应惊急，宿风旧楚衣。马上新草斜，此去何意兼取方绝年题灵州满亭.............................................外................................................


In [437]:
on_epoch_end2(32, _, temperature=1.3, maxlen=300, input_sentence="床前明月光，")

Epoch 32, output: 床前明月光，暮波去去尺。与醉丝经落，柳门不至移。白到衣锦起，依稀广晚霏。时生荆珠岸，幽莲须烟风。月风人犹似，虚情更拥长。别忆带诸物，立列堪更自。山芳叶不夏，西凉是罗书。东宫笑待节，忘见连洞衣。心上仙朝殿，未事别亡埃。一从合晚风，星象烟波香。苑眠陶映笑，一元别不还。门移情经一，朱海便即童年疏，杨柳章自雨浪度。舍鳐四何处，惊莲与玉机。时生岩宿散，浮种绝后声。巷从小神泪，久缘明我寻。沙长分共金水弄，恩此到近阳。桥千孤冷来，茫茫爱摇王。开解黄河意，繁兵水，秋。看手应忘久，（谢过外开。学逐莫阴乱，开窗本此书。孤果且黄野，忧僧无轻时。龙飞犹偏虽，为悲满更马。更见初迟下，归去与华愁。中长唯食日，携留笔江看。


In [439]:
on_epoch_end2(32, _, temperature=1.3, maxlen=300, input_sentence="君为镔铁吾为金，")

Epoch 32, output: 君为镔铁吾为金，多复将待已后儿。古木得云时与居，好思仙风轻得船。九宫芳教广十切，已复下剑一向来。对侍始食有千思，还泥怀帝镜枝霞。齐人动边肠有最，龙金自飞入都虚。才鸟不见共待此，可怜自在登何吟。老路向地鱼芳面，重徒数章秋风波。青松惊定一声闲。遥兹柳拂群方解，六音疏山可惜年。垂啼同花岳月竹年飞深处烧师罗罗洗粉边尊溪几流绕临喧，轻残路上通存暮荣宵老丹失昭一楚雨。迥被回猿经里无知言神心急路魄白色，闻镜皆相车。星海心程沉，中入饶雪绿。千千犹是还，壮惜照岸蹄。旧燕贫天国，山蝉碧阁新。药眠下交摇。朝忘唯开孤园微。放泊心使摇落穷，愿怜我怪风颜树。旧浪六虽终何止，苔，为足一根绿。待凤时云绿边晖，身烟少年一日忧。


In [444]:
on_epoch_end2(36, _, temperature=1.0, maxlen=300, input_sentence="今夕是何夕，")

Epoch 36, output: 今夕是何夕，秦城五只知。会回还病林，低长独无不。在辞紫烟镜，闲时无路太旧林。人何鱼节悲，将说去不心。时当逐云，南风满陵。秋风日初梦，堪从静古南。别后晴河酒，禅顶入云宜。居曲战一雨，坐宴含幕叶。知今爱有来，怀静一夜君。愁静万里照，断是山莺酒，楼高心思自。云水雨，红颜第，孤台一夜万。吴飞古来燕，林中鸟道开。宫皇半落省，霜明多人人。光惊故人愁，声是相在拟底何。舟羡心心物花光，情知草堂万里阴。几和君书梦相归。不知宫上乐，早妆梦中流。相思含封头，风波郡天尽。春北山朝待，种君后一是。小水后西里，凉气庭下疏。闭松断利面，分舞寂寞晨。犹来不能思，喜恨书前知。作礼接骨节，灵西十二去。北人茅催在水闻，寂寞相问年应


In [445]:
on_epoch_end2(36, _, temperature=1.3, maxlen=300, input_sentence="流水落花春去也，")

Epoch 36, output: 流水落花春去也，寒剑南归栖雁香。是随南君惟中少，莫说更是映内江云吟尘阁，城路共人彩光落，相似天心至地东。浮临春川带陵诗，曲声空此看，持始风外发。持江灯静养，盛后霜山坐¤堪遇喜策林栽清，寄处门木事天客。白死为宴意，疏池独转沉。十条池幕绮郎复不梦，孤华在高塞树景，应是朱见百夜间。西歌沈水黄木荣，正朝一声道处新。月当今从对柳与，嗟重残是海成亲。仙期玉作喜灵知，天明小酒醉忘春。塞落时是唯，殷勤语有音。意从飞闻小树鸯，章在也愧。乘真杨秋条，高怀杳寒月飘是，玄说连不卧马行。泉落更阶傍剑丛，朱避思应画逢枝。龙皇尽非自筵机，浪一回向学未春。野处又郎上一青，谁见蓬尨少.起。（《沈路金尽发，醉是未知十年闲。红花移


In [451]:
on_epoch_end2(36, _, temperature=1.2, maxlen=300, input_sentence="君为镔铁吾为金，")

Epoch 36, output: 君为镔铁吾为金，时有曲说影几里。春花衣堂更原法，金海尽名雪来晓。争能管乡争悲暮，凤露独梦静得日，滴亲思王秋阙傍。远知寻松满云日，苍茫千木常百处，寻几楼边玉霞尺。把花月照偶在露，身在露履老添雨，千在相逢独绕夕。影重世冠醉，分云未得在。最因人何不忘意，露传风识今光空。松堂且无星，三飞为思山。露徒从水少，杨柳独夜静。自为无时鸟，孤高虽已幽。古游长奏无行径，山垂四来满纤中。马蹄中上须，落花眼前平。争为难路柳，复鸡势离旌。自堪山上度，计作早傍行。清卷御天影，苔窗爱子池。多为亲经病不独，自是凰落穿栖过。上向美处今何。长赏争雪含后里，露思我日自不开。芳生国相醉风香，朱楼多知事画宿。和鹿不知掩清语，忽是未知亦


In [454]:
on_epoch_end2(52, _, temperature=1.3, maxlen=300, input_sentence="流水落花春去也，")

Epoch 52, output: 流水落花春去也，女头可屋往里非。树绕市处空业映，岂作金前更难忧，此夜生尽三水阴。丹竹寒门琴船灵，流向良叶未为稀。伤心一觉垂花月，珠壶有思双寒时。那须如师千湖绝，此生仙春任游痕。杨柳虚风飘飞树，关桐啼王千头壑，步歌红花到帝生。荒卧好风千吏南，昏移春鸟花报珠。明落空荷燕直禽，步主红心上时同。度斋金发斜叶引，槛喜风青。尘土双碧江青春艳未天绵欲秋井日月。水上解教园下见，销歌东北须傍绿。州程谁不方急啼，梅尽人意五人岐，远记隔洒国足露含主，，泉何时能合逢下云里，吹汉因窗满，看闲起结觉。叶旅重明归暂分，花香竟世水闲诏。醉阴何处来临流，朝是数山池边池。兰光无雪看玉侵，柳君有歌一晚微。那堪凉深游御却，暮云远觉空


In [455]:
on_epoch_end2(52, _, temperature=1.3, maxlen=300, input_sentence="会当凌绝顶，")

Epoch 52, output: 会当凌绝顶，修乱两夜经。机下平回侧，隐君会修生。莺名岁气休，半求望北凉。乡棹此归郡，新待岁月丹。程然能列绝，事岁山里行。取，侵月点，钓销待此江。处处白云州，太尽绝石阳。魂新更看王，大名色鸿春。梅花馀死有，乱云休居寒。浦舟既离书，柳被青叶残。寒逐竹德下，得珠向江蕊。言歇石临同，玉弓又初顾。关九旌旗酒，火相乱向林。梦传今绣教梧，怀僧前说云枕。可山莺归烛，横水远带村。罗动平光醉，波弄红尘荡。尽物虽与非，侯家随语心。春风岂今有村，层千气点秦。照收散为本，世叟朝人船。使君留生自，太守远望桃。古寺临清直，绿天接之回。收能暗小动，县边故短叹。相一未月蝉，士中白日长。发交愁深倚，师子旅客情。生花不不眼，贤依帝


In [456]:
on_epoch_end2(52, _, temperature=1.3, maxlen=300, input_sentence="生")

Epoch 52, output: 生来年不师上中，燕梁虽郎念斗营。直思每度沙百台，野便洞庭同朱池。双衣与风虽伤重，山月何石尽家时。晓一尚暖彩，久家仙楼几时香。僧君犹泛清州下，江云川极谷烛移。远，似风多两塘一吾，古雪怪入作蜀阴。苔烧惟道寒乡滴，仙国剑阳望至云。皆是已是长薄时，移金伤青阁晚角，弦出今马印苑红。死稀如赋无住垂，雨日今朝坐至绿。明月上有从应之，诗之今生太青山。早因红发乱歌鸟，著因翻至下玉逢。住得尘无天高影，作处云花在登中。北花军钱一仙才，台名王处绕今生。八上时冰病几主，浅犹尚王何百身，处处犹是识好城。阳台垂水迎雷两，浅烟郡色即此霜。还每诗家谁此气，野向侵田将主八。斋门养说过境后，墙小心泛公陵赤。泪向若闻与东秋，戍节看山


In [457]:
on_epoch_end2(52, _, temperature=1.3, maxlen=300, input_sentence="自古逢秋悲寂寥，")

Epoch 52, output: 自古逢秋悲寂寥，栖遥难是吏景居。病雪百色伤容黄，看是岁叶却看花。蜀马珠烟孤尽断，独无省处是龙川。可得新边掩绿看，苔畔移去入山迟。在家临断远美社，危入兰下尚故归。醉中月明行未腰，色日珠汉驿路空。最馀风尘谷幽楼，春南几长酒鹿边。待二动斋幽泉绝，病业终不会周贫。官车暮情新何节，能风愁年五主音。城闲伤至穷乘春，衣嗟壶时空江马。非处待诗多，亦望多逢心论船。秦楼向卷尚新弄，先生隔烛鸿石华。出里贵留春不意，丝人重阳中后师。几龙收杨笑传一，情下幽尽半下城。教舟龙吾近不话，怀逢，同望绕有，宜犹信枝多更莲。心梦凝尘菊盛开，荒向轻如夜中湖。从怜抱章晴前寺，持隐御云前负迟。马知能绮忽独同，送家荷鬓觉犹闻。谁忆此生愁要


In [459]:
on_epoch_end2(52, _, temperature=1.1, maxlen=300, input_sentence="春风得意马蹄疾，")

Epoch 52, output: 春风得意马蹄疾，登角长夕草色微。何人满衣丹夜雪，寻昏近带草禽醉。已闻使君谁相思，梦多青楼紫可出。销愁已闻翻旅语，禅和带雨玉外重。古木暂之邻青花，有有秋天衣上天。独吏无人行竹转，为知犹是马如闻。坐看若通主酒飞，病是使君从之多。弦者台上空罗身，井色风馀在轻故。暗雪背战想觉行，洛头银江自玉炉。子合秦手营水事，雨水更衣在合池。岂因无限命，迢递相对五春稀。应是愁多侵北晚，出得青人满城东。忆尽年二言闲。五头田园，知时三月，旗地云似，行林清念。年年新尽，平前南明。春雪天东，落落一合，之树未将。庭尽方已。竹把衣食，山阁千林。夫处出下，望本一与。万日满意，京月御日。叶向有可郎，相是却一稀。夜坐升问木，人馀见尔事


In [461]:
on_epoch_end2(52, _, temperature=1.1, maxlen=300, input_sentence="兰之猗猗，幽幽其香。")

Epoch 52, output: 兰之猗猗，幽幽其香。知被花暮，临去青山。岂是未尽，自别树听。为有诗尽，要神向日。思君鸾如，指岁若路。到日任玉，风足识.。秦.既当，言音谁成。一友瑶江，雪夜数思。自年珠朝，白头新阴。高想何事，识之乱青。露尽上，惆花多无。胡稀周上，前勤易是伤生。同心荒向绝，香动向御游。村长归，泥侵于，风襟清在春黄玉。文被入轻路，行喜临松重。知恨与忘来，百残过人津。鸟花相下归禽下，梅花墙南蝶心迟。不知玄随春不尽，日色古家无涯秋。轩须清明真牵心，苍山万花离谷阁。楚水旧有天不到，边泥知伤生宝书。直遥静色一千带，吟秋隔楼出乡声。馀生巢寺堂，星面应须绿》冬，还人窗此尽有情。恩梦此夜高春日，岂向满下少炎寒。为缘渔阳啼带自，书


In [463]:
on_epoch_end2(52, _, temperature=1.1, maxlen=300, input_sentence="凤凰于飞，翙翙其羽。")

Epoch 52, output: 凤凰于飞，翙翙其羽。龙落不有，影出犹出。海龙犹念，玉洒愁轻。鹤去弹言，卷马钟叶。何以已轻，力亦无事。河光多以。百日寒桑，难著几条。迥见入面，花有因登。有时飞人，楚水三十。御心一花，暂前天上。台罗池自，四明将此。心山残烟，一生龙路二。神仙新且接，出，此时水。景天云秀声，又主无守处。四寞复斗冠，官君隔化苔。殿.来晓去，华卧几恐时。楼山此空缘，荡.尽天身。今时未知身，国马送士时。风月未动潭，文可残桐杯。惊老第，驿通台。与君黄玉海，桥似要子机。手金行堂火，禽林树筵声。同塘动野树，食新玉是清。出色出怨少，者勤雨起深。初岁话白，山出不以。通知五华，平生马为。空倚三行，日落风作。知.思灵，火雪临北。古马清倚


In [465]:
on_epoch_end2(59, _, temperature=1.1, maxlen=300, input_sentence="凤凰于飞，翙翙其羽。")

Epoch 59, output: 凤凰于飞，翙翙其羽。亭四欲远，郊阳天中。年年不可素，却巢相思寺，长银悲为看。每知素入灭，愁中动始城。有中诗稀花，愁寞山莫逢。遥怜与文水，通目衣欲今。斋深留虽醉，千头夜酒香。春风留吴时，著家上秋机。香时数片鹤，愿笑心如塞。岁心南头夜，风青卧书家。终日无苦郎，便带石关次。细草负古梅，香香影晴倾。何见倚江远，沙酒寄物晖。还为《惭香，无劳在相密。忘书乐鲜堤，为雁多精神。灯下青门过，青人出水新。知为吟诏向，声陌待天存。西山天有河，南当远之归。南宫天天至，林怜思非数。青花药通郎，代年却栖闻。天何听烛寻，离采将白。主野下童。日绣野异晚，桂霞何出还。色横无杯心，相见满秋游。莫波树有尽，日观事在众。清关春雨节，


In [467]:
on_epoch_end2(59, _, temperature=1.2, maxlen=300, input_sentence="兰之猗猗，幽幽其香。")

Epoch 59, output: 兰之猗猗，幽幽其香。疑盖真枯，西风百叶。久峰与此，春神知。离人处。晚君生三出，六去如何归。古海还开钱，听侵如将天。渐带《林，前心风流仙。吴人不忘病，知如开中下。万师游云日，夜闻都知起。夜生有背得，远近日南目。碧草多神时，风树遗天夕。自拟寞已师，浮衣已还生。乱目头枕水，金院万里县。为南岩色当，终夜酒下流。斋秋来在得，起以方不结。青不能秋去，越客阳中文。无郡千里日，独水送天门。黄河深浦径，高主似人暮。山下历不见，下交离隔出，殿如岂岩忘，吟花鸣远寒。八月天流极，十为思无东。其城已悠日，长阳金多穷。下骑长花药，斜飘户清好。十边前火处，去关心少见。阙扉峰何间，长当不秋丝。无清流过近，开条天路秋。云河深花


In [468]:
on_epoch_end2(59, _, temperature=1.2, maxlen=300, input_sentence="春风得意马蹄疾，")

Epoch 59, output: 春风得意马蹄疾，北家圣处斋隐碧。楼阳紫雨应在白，城馆独闻策来行。积年楼上和风此，金根翠见谁教金。久闻堪花色。不须游，乘凉下，北条有子初出楼。空中雪深渔（鹤，始得归时白江流。重里名将神酒锦，荒，朝日是游生。画将知思》）三...............................................................................................................................................................................................


In [469]:
on_epoch_end2(59, _, temperature=1.2, maxlen=300, input_sentence="流水落花春去也，")

Epoch 59, output: 流水落花春去也，作思书步客红池。悬惊春水书下尽，金阁如为碧秋寒。大上水象青水落，岁月关作风暂轻。如今泪多心一忘，独遥吾向碧相回。画有疑陈犹尘栽，莲花不是今日琴。夜看啼来边难宅，青青休来欲何人。两门日入游采客，君思芜泉入梦回。傍满天门寻伤怜，待世楼中自何须。云独疑死浪令成，潮林金起在朝面。新酒文行能自乡，风寒金衣子头栖。碧将山于人上阳，从此书气得无折。一初暗催看丹枝，无长莫思水欲留。翻情独对功事此，事官待中凝便月。远关思清渔众景，连阳回者送泥城。斜晖莫间昼萧光，地深松风一林盛。流年形丝盛帆望，暮花梦声岩紫云。绝家能惊仙地发，得意回为久依长。江南唯知深惟泽，称君杀花有下寒。独事才.俗能衣，孤金梅花


In [470]:
on_epoch_end2(59, _, temperature=1.2, maxlen=300, input_sentence="今夕是何夕，")

Epoch 59, output: 今夕是何夕，银丛惜应唯。清袅分时已可会，秋将鸟驱书。唯难入幽羽，泪叶去三流。神未轻望过，云从白宇风。甘宜添起药，晚愁出心村。更自上上少，见黄才日调。应百多变苍，山作远云名。宝诗去不卿，采水怀毛中。风叹行客好，寒棹玉水臣。落日满九山，（会丹江风。风流从前去，烟天寄仙岁。人将游九水，莫然霞空风。终日鱼不时，人情飞秋色。当年点身贫，笙间发几沙。月爱寄丹出，风月待歌多。今日山下远，老树风更屋。寒阳想凝下，千门满湿夫。斋叹身惟作自须，北居王门神。金塞嗟将送马涛，名衔师千休。只知入北客，古鹤深八人。日如引帝绿，须学凤室汉。水巷满茫门，新龙上东南。草歇若何尽，万荷初此目。当来方地病，拜斋浮雨泽。竹气觉似雨，


In [477]:
on_epoch_end2(124, _, temperature=1.2, maxlen=300, input_sentence="兰之猗猗，幽幽其香。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 兰之猗猗，幽幽其香。袅袅逢寒，清君不奏。官酒白夕轻摇扇，江色是真卧翠回。四朝江月作地宿，双川卧断出楼前。异人怀将分恩闻，与鼎天上转如阙。花远人年会物楼，只岂登鸣如戍人。金垂罗馀，吾禅惟井师朱台，送子桃暂未断宁。隐书应是主梦游，君子愁奇过虚寒。头林夜到皆通罢，书冠允黄绝上金。园山皆去妆送壁，迟迟亦神别事山。朝容虽旧几功，双松下亭，尘语凉冬凝绿鸡。四果青洲吟为烟，欢数有先同言情。秋来金结绿向，分园取子足朱色，越见由香送酒翠。盈门秋阳天子极，马堪金俗水林月。文意凌来魂峰处，香知遗在驱翠空。容桥安处天风梦，心别垂人常垂作。既无野中兰渔霜，海后万山车子鬓。神降一仙绝归色，霜被达子月雾中。旧明得时旌落太在


In [474]:
on_epoch_end2(124, _, temperature=1.2, maxlen=300, input_sentence="春风得意马蹄疾，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 春风得意马蹄疾，金蝉歌明作分空。良所应见光北色，东岭万车欲使身。落日万物舞满路，花齐云边春犹轻。衣花落月行风光，黄石万门变芳骨。更深天子孤衣金，静当莲上依茫茫。飞草紫方若和夜，放声知身一正回。一说新熟，山离五雾。空草人非争犹，相悲犹。鼓鱼高楼当物，宅所岸，人端唯清处，微溪青茫，亲定，旗成擖断和山下。尽凝不足是僧力，何处虽终何日河。却寻投鸟宝。容边悲会欢。惟将归有青寥时，晚流有地有无心。无有树圆高女所，生者今莫正精未。披河虽处分无语，方遥风畔乍残碧。中处欲书弹书院，杨花一枕朝盛身。木乡求家即来主，金轻万树许摇来。水真全主去相思，繁声寒下深楼声。雨同烟景寻客城，高为征菊，狂曙临梦笛，望夜应树强上语


In [475]:
on_epoch_end2(124, _, temperature=1.2, maxlen=300, input_sentence="流水落花春去也，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 流水落花春去也，丝是钟女以万二。谁喜玉馀贫相同，都无年貌是后时。我归日落枯复感，当时林水赏字绿。临暮暮吟犹有药，流门当处动鸣飞。狂风白树行波阳，黄微看地无风山。知情多处花凉节，尽向青山管女晨。功秦一别深楼室，天子白骑自常微。沙方栈往凤年日，莫大盖之隔落阳。双栖尺竹花有马，终情偏望花闻真。当朝多尽，八报早槛有阳，逢年西处行回笑。怅望寒草天门小，愁带三结散家阳。闻说千翠醉空古，红暗因家月风雪。传两当鹤肠带雾，窗中未长不子秋。再夫客断断分欲，一卧常把君无新。（见诗年须晚开神乌斗寄看歌。石作不看翻常轩，阳王不识宿声蝉。古园生尽无息女，亦在有常梁东时。庭头空色长独岳，池中忆天对香闻。重各洒死绣波非，城土


In [476]:
on_epoch_end2(124, _, temperature=1.2, maxlen=300, input_sentence="今夕是何夕，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，只天光上汉。沙州沧阳阳，双芳荷未身。古临无见足，昔年中草园。游雾落如手，归闻拂岂落。阳入参差红，心去暗贤能。闻将起神色，秋庭拂安国。楼树台明谷，馀浪无幽诸。拂霞此秦。岁月与尘颜，回间头未明。看人闻与户，残涧方归将。独从殊上路，岁酒风景断。半底秋峰池，带人夜欢驱。笛久卷时接，只为带万生。古往从玉力，神尽生自歌。烛边未胜女，阑临水理冷。万照寒目笑，乃须登锦烟。玉青东偏薄，南山失乡世。坐看杏花凝，关生成说野。古来翻异梦，归来未见叹。古木太玉汉，千重萧萧条。知里宿神意，拟缘无心来。依然作以酒，性道云舟处。知书千草水，性贤日明名。岂共两岂主，三生中一边。菊乌日妆游，荣明必北时。斋云深外达，一


In [478]:
on_epoch_end2(124, _, temperature=1.0, maxlen=300, input_sentence="今夕是何夕，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，尽自吹车绣。知人孤面小，人家时旧扇。古人可解角，湘陵心日霞。古天秋草静，虽深有为歌。馀海相知涯，一声谁有书。上君秋夜暮，一深学气西。魂西不胜思，云树犹萧。杏但拂心吴发。应能海上临峰兵，阳中落上时翠寒。此时看迹轻馀游，别亲万家来空夫。不向夜孤两径出，衣期仙作月景书。野笑白莺小梅柳，胡上先依春夜知。杨柳若知须林夜，声往不他随白云。六浦花树天下客，望河愁马入中玉。我在与寄闭眼闻，今年如在隔山楼。残动唯炉来一客，犹应日暮寄别人。作神憔湲全谁不，难将何处处处生。一物三心少年老，雪断寒间吴海足。九龙生客无穷目，素土入收乌龙声。...药醉舞。儒，晓亲归笑，庭边年日南北行七吟两。红头依守，惟心长然


In [479]:
on_epoch_end2(124, _, temperature=0.8, maxlen=300, input_sentence="今夕是何夕，晚风过花庭。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，晚风过花庭。云色谁王雨，云影高歌自。白发欲成水，云多行雨时。同年三雪水，女物山山泉。偶将门鸟来，云高南后商。古木松下客，不见时家游。孤城风雨尽，野影共登南。芳草犹往惟，之尽不岳初。日月曾灵景，多旧万星林。骑事萧萧然，天天门未空。山水已经月，泉波立此回。古来逢作目，犹当养别离。谁知逢若惟，江山出一情。荣当必化容，回国终已秋。玉楼能道宿，行与忽化者。却方花上出，归来白鸟来。夜深云雨尽，西行能宿良。为师生城尽，山泉日月多。古庙黄著影，三发送穷发。云水未宵是，烟花客亦同。古雷行何处，春深落寒色。谁逢红树尽，地秋满戍露。还似未回色，未床与高发。谁教送光主，清人中未秋。岂无江色目，方车穿色尘。


In [488]:
on_epoch_end2(124, _, temperature=1.0, maxlen=300, input_sentence="今夕是何夕，晚风过花庭。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，晚风过花庭。谁知白陵柳，楼台语夕不。生在且霜发，深王旗穷霞。散洛望芳色，醉之出天水。春宿看横遥，红欲流河看。烧无可所断，老声黄悲绕。不知在尘色，尊前明月流。主人池上中，应随彩青深。客人何处客，云帆望浪长。谁知不回药，晚望暗宿沙。杨花草木老，玉门来雪开。游时风未深，天河犹几雪。知见欲起好，有此忽人才。故映日年人，行人星与可。今日既一寺，此长虽何一。琴林须晚镜，老病月阳空。客寻不车白，骨今弹老中。临书万寂寞，只应应欲新。七闻求吟朝，满落今何年。门静景闲日，青自门征颜。不见门夫，处处得归馀。影拂且异径，旌旗独江淮。暮吟知往每，半为心两玉。凉花不知公，至今与方是。今日辞颜，门宿湖上下。客看


In [493]:
on_epoch_end2(145, _, temperature=1.2, maxlen=300, input_sentence="兰之猗猗，幽幽其香。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 125, output: 兰之猗猗，幽幽其香。锦百似松食，花初知作星。报闻山文上，舞名亦性宫。深戎知宜我，见然开未知。定心知悲浪，老远出背寒。北夜同数身，台路乞旧山。绕夏李山饮，子骨不旗枝。世子路半郭，未花亦同野。细色春长死，隐旧为仙来。知君无汉道，落花入半结。玉片秋石门，悬药方鸡我。云，犬僧人。玉老香行乐，远渡一白时。宴野方重带，那知台好山。同便朝当时，高卧叶逢心。落花唯负泛，连去夜正别。报山虏然贤，得外为归主。寂寞万里酒，夜接起灭归。三烧秋多在，几驾独多天。皇》见青竹，重嗟相散稀。浮清上霭夕，七里逐重步。尽绕水衰浪，声景去雁流。分市真泉浪，枕山人事取。林随子无对，庭帆青柳暮。二木云深梦，有繁几家对。大红老边梦，月吟


In [495]:
on_epoch_end2(145, _, temperature=1.2, maxlen=300, input_sentence="野")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 野馆三陵称殊桃，却近杨柳来公身。我思不回，和久无，七，孤凰之床。风生秋对欲烟平，王口何但行早还。闻风堤地大六骨，得书残峡自势空。笙歌正笑梧桐通，汉得百日念侯家。丹拟水色功难觉，闲家在北百未垂。投城万里对泥贤，志中连烧常五霞。闻得遥独论伤语，信在自为月半九。雪稀空复风海上，玉树题花白一种。枕阳既泪客面归，送室以檐夜土书。圣女故漫吹香重，横片阳时桥双井。数落重华八候自，承古落烟尘。（钟九月使燕香上。子日石绿高舞且，终风不觉碧流人。邻石黄金行侯日，山中中夕是轩然。太边从马朝明地，成故寒门昭绮衣。闲游既客西过竹，千识从法碧光真。被华生徒洞歌郭，吴花独望书春时。马发自惊雄路对，偏持乍成共月将。细.更结催


In [496]:
on_epoch_end2(145, _, temperature=1.2, maxlen=300, input_sentence="流水落花春去也，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 流水落花春去也，便向留惊晚早行。吾有涯寂碧于青，独去香为花峰寒。世人上堂黄楼梦，酒用里翠不须身。皇尽花行黄七。，更见遥但不将得。戎阴老之忆。出戍未仙欲成烧，飞涛徒当含芳归。时须图有闲庭已，秀中相随暮后稀。盘色白发之柳面，不向来冥养去年。病复未惊，丹汉朱金王知方方属落。可问到衣非如燕，繁井前子六五馆。战之流泉才醒语，晓间不动闻来秦。问门女成凤人年，何凉失山花华可。时席花雄紫里定，为道花分虚游期。竹湘无地五花寺，谁是我分闲宫南。坐看所天断地。，半入因年信千秋。旧风看重游报神，子情春起自看去。客深洞松称恩如，登山诗家马归丹。沈梦无无桃昨白，忽闻何处秋雨乘。翠镜清汉轻图杨，与子中事身多时。檐须朝泪绿柳


In [499]:
on_epoch_end2(145, _, temperature=1.0, maxlen=300, input_sentence="今夕是何夕，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，远物亦笑合。古今静分合，道入欲舟斜。云空共却倾，萧条出上横。知德依绝鸣，向日石应，日..，竹上人。圣君启中，万限仍愁。归去如依闻翠重，天涯流石仙面章。云意四重闲安尽，家心觉雨动几相。一夜四境野秋雪，才香逢梦人立重。使帝云愁云景别，家时朱人岁世人。知乡山此奇旧，无期。得处别成中，唯犹春侯秋。语岁山浪月，引入风动深。西里经山夜，尽来四远心。无馀水霞来，琴殿路目圣。春风无家艳，孤台一寂寥。高鸟入中，................................................................................................


In [498]:
on_epoch_end2(145, _, temperature=1.2, maxlen=300, input_sentence="今夕是何夕，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 124, output: 今夕是何夕，离门照未开。文章时时老，流水欲高风。短行乡怀海，浮床亦送笑。青玉与常老，可将白露难。梦真神用后，诗酒爱纷纷。封节良来尽，青腰南隐王。住来乡程至，岁即杜帘灯。峰所上州合，今已上人间。南户昭叶池，不自出树垂。知子满长路，影有石拂岩。诗自白发远，同情粗难平。立自一无信，何之里亦黄。知《池路旧，何处白云归。何处宫气道，种得总怨翠。墙衔中芳山，名秦屡池斜。遥含知乐晚，摇斜香亦阶。楚叹流天住，晚怨留羽亭。应缘馀平鹤，绕此夜凝魂。可怜少石曲，寒光行复游。所见似卷兴，风流含城室。别心行生语，兼桥渭落迟。欲如如剑飞，七人带平曙。只因乞如我，春心三十家。惭生僧期主，忆堪落池丹。便《故马心。鱼道正外响，


In [500]:
on_epoch_end2(145, _, temperature=0.5, maxlen=300, input_sentence="今夕是何夕，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 今夕是何夕，朝来似倚楼。知君无得道，秋风老路长。日暮人无处，一共几年诗。山路无传宅，龙时夜亦归。知子无言事，今心独行情。知将人名来，花路一。斋，时来故竹山。不知仙归色，夜身上树归。知间无梦频，天涯望山前。知须独无老，山中正轻行。（《人无时，机心不无时。知须如梦听，流水流山琴。时时春路断，高山欲远秋。知须秋无路，不知一路时。不知秋流隔，长歌不成知。南轩愁神酒，一时自成笑。知家事中路，不知亦成时。知心为我夜，高树亦成神。知子生成事，时时白月光。古人无亦事，玉楼欲春游。知高流水里，春来花火来。知言三十里，不如不自楼。知声欲无路，不知欲远书。云路长功我，正水落成时。风光尽无去，高高人间事。年心无难广，


In [507]:
on_epoch_end2(145, _, temperature=1.0, maxlen=300, input_sentence="今夕是何夕，晚风过花庭。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 今夕是何夕，晚风过花庭。何年成圣水，南细相香愁。石高独多路，东从大石门。花里难细马，花潮见路情。待人共罗里，应传柬草事。前容游取路，凉雪如何如。峰郎独稀小，霜图少寒风。欲逢高鸟湿，高低别浪城。凄凄君后远，长朝动待斋。回时成语净，开微失路鸣。清飘苍好大，暖晚半声喧。低露飘殿鸟，四来满方户。（《雪亦时，拟河为恩来。主人与古日，身路朝时声。今日初夜转，四年无时人。背鸿仙云歌，酒华当仙云。中过归成徒，露中古此十。带来满光度，黄露入将草。日日常未断，洞门双至阳。春老无常月，秋景欲竹微。强游闻潮客，夜舞万里深。马居皆衣色，两连业倚载。野地宫几分，多病云变方。流溪夜并为，近得有平唱。主人乞无时，青山色塞白。


In [506]:
on_epoch_end2(145, _, temperature=2, maxlen=300, input_sentence="今夕是何夕，晚风过花庭。")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 今夕是何夕，晚风过花庭。有霜摇灯晓，端《猴水道。有晨，唯借既合，夜火自明静蜀足衣歌道嘉落。相秦如笔江，观浪此急绝乐官，逢霄寄悠便屋草。塞径非种天诸主，今路藤劳秋朝秀。灯似嶷陇字弟。那磬秀境极能汉恐野节食枯作可车轩他我空千粉能轩子画房馆开皆兮正人更¤往影，能合蹄秦舟漏归闻川间帘依朝指，可忽闲《两夜灵，江壑致县名逐传。史客乍青军声水，童数令有战且却。银思非鼓袖曙事，叠花白首公底来。结山寺我看薄邑，归地秋浦德有元。出今一殿之寂壁，风容雪远似高命。掩静重泉笼君事，华身迹玉迷客乐。郭连当岁当山老，偏缘难寻花未寂。津真歌身悲自会，好人声潮荒风他。气府湘桐门本玉，悲竹道书老死窗。欢怀鹤来花忆得，过必如兹陇风


In [503]:
on_epoch_end2(145, _, temperature=0.1, maxlen=300, input_sentence="今夕是何兰之猗，")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Epoch 145, output: 今夕是何兰之猗，高花流处一时归。知心不是，谁人便，不复无时，神，谁人。不知不复，三沙野之。知有不成，七时无。知名不复，四浪两。知无人尽，四成往。日，砌，花花人。知君不复，三成我能。知名不复，三成我容。知子不成，七时无。知子不成，七时无。知名不复，三馀我能。知不不成，七时无。知有不成，四边便，不复无心，三成我能。知子不成，五，谁人。不知不复，四成之花。知有不成，七时无。知有不成，四边便，不复无时，神，谁《三容。不知不复，四浪隔沙。知子不成，七时无。知不不成，七时，不见不成，谁我无。不可无成，四成朝，不可无时，三成我容。知名不复，三成我我。知子不成，七时无。斋来不成，七时无。斋风不成，四成惟。知不
